In [1]:
!pip install git+https://github.com/KindXiaoming/pykan.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/KindXiaoming/pykan.git to /tmp/pip-req-build-lyiknes7
  Running command git clone --filter=blob:none --quiet https://github.com/KindXiaoming/pykan.git /tmp/pip-req-build-lyiknes7
  Resolved https://github.com/KindXiaoming/pykan.git to commit 00a9077100b9783304b1df95de47b7344e3e1eec
  Preparing metadata (setup.py) ... done


In [2]:
!pip install torch matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [6]:
pip install scikit-learn==1.1.3

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moabb 1.1.1 requires scikit-learn>=1.4.2, but you have scikit-learn 1.1.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import matplotlib.pyplot as plt
from kan import *

/home/kunuru/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [4]:
!pip install mne
import mne

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install moabb

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3


In [7]:
import warnings
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
import mne
import moabb
from moabb.datasets import BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery
from mne.decoding import CSP
from mne import Epochs, pick_types, events_from_annotations
from kan import *
from kan import KAN
from kan import LBFGS
from sklearn.metrics import f1_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from torch.nn import Sequential, Linear, ReLU, CrossEntropyLoss, Module

from torch.utils.data import DataLoader, TensorDataset
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')# if torch.cuda.is_available() else 'cpu')

import torch.optim as optim
from tqdm import tqdm
import time
import json


epochs=20
batch = 16
lr = 0.1
loss_fn = CrossEntropyLoss()
steps=50
kf = KFold(n_splits=10, shuffle=True, random_state=42)
subjects= [1,2,3,4,5,6,7,8,9]
le = LabelEncoder()
hidden_layers= 10

moabb.set_log_level("info")
warnings.filterwarnings("ignore")
csp = CSP(n_components=4, reg='oas')
def data_EEG(subjects, train=True, multiclass=False):
        if train:
                session_name = "0train"

        else:
                session_name="1test"


        dataset = BNCI2014_001()

        if multiclass:
                event_id = dict(left=1, right=2, feet=3, tongue=4)
        else:
                event_id = dict(left=1, right=2)#, feet=3, tongue=4)

        tmin, tmax = -1., 6.
        runs = range(5)

        epochs_data=[]
        for subject in subjects:
                sessions = dataset.get_data(subjects=[subject])
                for run in runs:
                        raw = sessions[subject][session_name][str(run)]
                        events = mne.find_events(raw, stim_channel="stim")
                        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
                        raw.filter(7., 35., method='iir', picks=picks)

                        epochs = Epochs(raw,
                                events,
                                event_id,
                                tmin,
                                tmax,
                                proj=True,
                                picks=picks,
                                baseline=None,
                                preload=True,
                                verbose=False)
                        epochs_data.append(epochs)
        return epochs_data

def export_json(data, filename):
    for key, value in data.items():
        if isinstance(value, np.ndarray):
            data[key] = value.tolist()
        elif isinstance(value, list) and all(isinstance(x, list) for x in value):  # Check if ALL items are lists
            data[key] = [[float(item) for item in sublist] for sublist in value]  # Convert inner elements to float
        elif isinstance(value, list):
            data[key] = [float(item) for item in value]  # Convert the numbers to float in the list


    with open(filename, "w") as outfile:
        json.dump(data, outfile, indent=4)
class KANModel(Module):
    def __init__(self, input_shape, output_shape, device='cpu'):
        super(KANModel, self).__init__()
        self.kan = KAN(width=[input_shape, output_shape], grid=3, k=3, seed=0, device=device)

        # Ensure all parameters are in double precision
        for param in self.kan.parameters():
            param.data = param.data.double()

    def forward(self, x):
        return self.kan(x)

    def train_model(self, dataset, opt="LBFGS", steps=100, log=1, lamb=0., lamb_l1=1., lamb_entropy=2.,
                    lamb_coef=0., lamb_coefdiff=0., update_grid=True, grid_update_num=10,
                    stop_grid_update_step=50, batch=-1, loss_fn=None, lr=1.,
                    small_mag_threshold=1e-16, small_reg_factor=1., sglr_avoid=False):

        # Initialize progress bar
        pbar = tqdm(range(steps), desc='Training KAN', ncols=100)

        # Define loss function and accuracy metrics
        if loss_fn is None:
            loss_fn = lambda x, y: torch.mean((x - y) ** 2)
        acc = lambda x, y: torch.mean((torch.argmax(x, dim=1) == y).float())

        # Configure the optimizer
        if opt == "Adam":
            optimizer = optim.Adam(self.parameters(), lr=lr)
        elif opt == "LBFGS":
            optimizer = LBFGS(self.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe")

        results = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}

        if batch == -1 or batch > dataset['train_input'].shape[0]:
            batch_size = dataset['train_input'].shape[0]
        else:
            batch_size = batch

        global train_loss, train_acc

        def closure():
            global train_loss, train_acc
            optimizer.zero_grad()
            pred = self.forward(dataset['train_input'][train_id].to(self.kan.device))
            if sglr_avoid:
                id_ = torch.where(torch.isnan(torch.sum(pred, dim=1)) == False)[0]
                train_loss = loss_fn(pred[id_], dataset['train_label'][train_id][id_].to(self.kan.device))
            else:
                train_loss = loss_fn(pred, dataset['train_label'][train_id].to(self.kan.device))
            train_acc = acc(pred, dataset['train_label'][train_id].to(self.kan.device))

            objective = train_loss + lamb
            objective.backward()
            return objective

        for _ in pbar:
            train_id = np.random.choice(dataset['train_input'].shape[0], batch_size, replace=False)
            test_id = np.random.choice(dataset['test_input'].shape[0], batch_size, replace=True)

            if opt == "LBFGS":
                optimizer.step(closure)
            else:
                pred = self.forward(dataset['train_input'][train_id].to(self.kan.device))
                if sglr_avoid:
                    id_ = torch.where(torch.isnan(torch.sum(pred, dim=1)) == False)[0]
                    train_loss = loss_fn(pred[id_], dataset['train_label'][train_id][id_].to(self.kan.device))
                else:
                    train_loss = loss_fn(pred, dataset['train_label'][train_id].to(self.kan.device))
                loss = train_loss + lamb
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            test_loss = loss_fn(self.forward(dataset['test_input'][test_id].to(self.kan.device)), dataset['test_label'][test_id].to(self.kan.device))
            test_acc = acc(self.forward(dataset['test_input'][test_id].to(self.kan.device)), dataset['test_label'][test_id].to(self.kan.device))

            if _ % log == 0:
                pbar.set_description("train acc: %.3f | test acc: %.3f " % (train_acc.cpu().detach().numpy(), test_acc.cpu().detach().numpy()))

            results['train_loss'].append(torch.sqrt(train_loss).cpu().detach().numpy())
            results['test_loss'].append(torch.sqrt(test_loss).cpu().detach().numpy())
            results['train_acc'].append(train_acc.cpu().detach().numpy())
            results['test_acc'].append(test_acc.cpu().detach().numpy())

        return results


class MLP(Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.layers = Sequential(
            Linear(input_dim, hidden_dim),
            ReLU(),  # Activation function (you can experiment with others)
            Linear(hidden_dim, output_dim)  # Output layer
        )

    def forward(self, x):
        return self.layers(x)


    def train(self, dataset, opt="LBFGS", steps=100, log=1, lamb=0., lamb_l1=1., lamb_entropy=2., lamb_coef=0., lamb_coefdiff=0., update_grid=True, grid_update_num=10, loss_fn=None, lr=1., stop_grid_update_step=50, batch=-1,
              small_mag_threshold=1e-16, small_reg_factor=1., metrics=None, sglr_avoid=False, save_fig=False, in_vars=None, out_vars=None, beta=3, save_fig_freq=1, img_folder='./video', device='cpu'):
        '''
        training

        Args:
        -----
            dataset : dic
                contains dataset['train_input'], dataset['train_label'], dataset['test_input'], dataset['test_label']
            opt : str
                "LBFGS" or "Adam"
            steps : int
                training steps
            log : int
                logging frequency
            lamb : float
                overall penalty strength
            lamb_l1 : float
                l1 penalty strength
            lamb_entropy : float
                entropy penalty strength
            lamb_coef : float
                coefficient magnitude penalty strength
            lamb_coefdiff : float
                difference of nearby coefficits (smoothness) penalty strength
            update_grid : bool
                If True, update grid regularly before stop_grid_update_step
            grid_update_num : int
                the number of grid updates before stop_grid_update_step
            stop_grid_update_step : int
                no grid updates after this training step
            batch : int
                batch size, if -1 then full.
            small_mag_threshold : float
                threshold to determine large or small numbers (may want to apply larger penalty to smaller numbers)
            small_reg_factor : float
                penalty strength applied to small factors relative to large factos
            device : str
                device
            save_fig_freq : int
                save figure every (save_fig_freq) step

        Returns:
        --------
            results : dic
                results['train_loss'], 1D array of training losses (RMSE)
                results['test_loss'], 1D array of test losses (RMSE)
                results['reg'], 1D array of regularization

        Example
        -------
        >>> # for interactive examples, please see demos
        >>> from utils import create_dataset
        >>> model = KAN(width=[2,5,1], grid=5, k=3, noise_scale=0.1, seed=0)
        >>> f = lambda x: torch.exp(torch.sin(torch.pi*x[:,[0]]) + x[:,[1]]**2)
        >>> dataset = create_dataset(f, n_var=2)
        >>> model.train(dataset, opt='LBFGS', steps=50, lamb=0.01);
        >>> model.plot()
        '''


        pbar = tqdm(range(steps), desc='description', ncols=100)

        if loss_fn == None:
            loss_fn = loss_fn_eval = lambda x, y: torch.mean((x - y) ** 2)
        else:
            loss_fn = loss_fn_eval = loss_fn

        acc = acc_eval = lambda x,y: torch.mean((torch.argmax(x, dim=1) == y).float())

        grid_update_freq = int(stop_grid_update_step / grid_update_num)

        if opt == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        elif opt == "LBFGS":
            optimizer = LBFGS(self.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32, tolerance_ys=1e-32)

        results = {}
        results['train_loss'] = []
        results['test_loss'] = []
        results['train_acc'] = []
        results['test_acc'] = []
        if metrics != None:
            for i in range(len(metrics)):
                results[metrics[i].__name__] = []

        if batch == -1 or batch > dataset['train_input'].shape[0]:
            batch_size = dataset['train_input'].shape[0]
            batch_size_test = dataset['test_input'].shape[0]
        else:
            batch_size = batch
            batch_size_test = batch

        global train_loss, train_acc

        def closure():
            global train_loss, train_acc
            optimizer.zero_grad()
            pred = self.forward(dataset['train_input'][train_id].to(device))
            if sglr_avoid == True:
                id_ = torch.where(torch.isnan(torch.sum(pred, dim=1)) == False)[0]
                train_loss = loss_fn(pred[id_], dataset['train_label'][train_id][id_].to(device))
            else:
                train_loss = loss_fn(pred, dataset['train_label'][train_id].to(device))
            train_acc = acc(pred, dataset['train_label'][train_id].to(device))

            objective = train_loss + lamb
            objective.backward()
            return objective

        if save_fig:
            if not os.path.exists(img_folder):
                os.makedirs(img_folder)

        for _ in pbar:

            train_id = np.random.choice(dataset['train_input'].shape[0], batch_size, replace=False)
            test_id = np.random.choice(dataset['test_input'].shape[0], batch_size_test, replace=False)

            #if _ % grid_update_freq == 0 and _ < stop_grid_update_step and update_grid:
            #    self.update_grid_from_samples(dataset['train_input'][train_id].to(device))

            if opt == "LBFGS":
                optimizer.step(closure)

            if opt == "Adam":
                pred = self.forward(dataset['train_input'][train_id].to(device))
                if sglr_avoid == True:
                    id_ = torch.where(torch.isnan(torch.sum(pred, dim=1)) == False)[0]
                    train_loss = loss_fn(pred[id_], dataset['train_label'][train_id][id_].to(device))
                else:
                    train_loss = loss_fn(pred, dataset['train_label'][train_id].to(device))
                loss = train_loss + lamb
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            test_loss = loss_fn_eval(self.forward(dataset['test_input'][test_id].to(device)), dataset['test_label'][test_id].to(device))
            test_acc = acc_eval(self.forward(dataset['test_input'][test_id].to(device)), dataset['test_label'][test_id].to(device))

            if _ % log == 0:
                pbar.set_description("train acc: %.3f | test acc: %.3f " % ((train_acc).cpu().detach().numpy(), (test_acc).cpu().detach().numpy()))

            if metrics != None:
                for i in range(len(metrics)):
                    results[metrics[i].__name__].append(metrics[i]().item())

            results['train_loss'].append(torch.sqrt(train_loss).cpu().detach().numpy())
            results['test_loss'].append(torch.sqrt(test_loss).cpu().detach().numpy())
            results['train_acc'].append(train_acc.cpu().detach().numpy())
            results['test_acc'].append(test_acc.cpu().detach().numpy())

            if save_fig and _ % save_fig_freq == 0:
                self.plot(folder=img_folder, in_vars=in_vars, out_vars=out_vars, title="Step {}".format(_), beta=beta)
                plt.savefig(img_folder + '/' + str(_) + '.jpg', bbox_inches='tight', dpi=200)
                plt.close()

        return results

In [10]:
# TWO_CLASSRAW
metrics_KAN = {}
metrics_MLP = {}
total_train_time_KAN = {}
total_train_time_MLP = {}

subjects = [1,2,3]  # Add all 9 subjects here

for subject in subjects:
    print(f"### Subject {subject} ###")

    # Get data for each subject
    epochs_EEG = mne.concatenate_epochs(data_EEG(subjects=[subject]))
    df = epochs_EEG.to_data_frame().drop(columns=['time', 'epoch'])
    X = df.drop(columns=['condition']).values
    y = df.condition.values
    y = le.fit_transform(y)

    X = torch.from_numpy(X).double()
    y = torch.from_numpy(y).long()

    input_layer = X.shape[1]
    output_layer = len(np.unique(y))

    kan_model = KANModel(input_shape=input_layer, output_shape=output_layer, device=device)
    mlp_model = MLP(input_layer, hidden_layers, output_layer).to(device)

    for param in mlp_model.parameters():
        param.data = param.data.double()

    kan_optimizer = LBFGS(kan_model.parameters(), lr=lr)
    mlp_optimizer = LBFGS(mlp_model.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe")

    metrics_KAN[subject] = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'test_loss': [], 'train_time': 0}
    metrics_MLP[subject] = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'test_loss': [], 'train_time': 0}

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        dataset = {
            'train_input': X_train,
            'train_label': y_train,
            'test_input': X_test,
            'test_label': y_test
        }

        # Train KAN
        print(f"--- Fold {fold + 1} ---")

        fold_metrics_KAN_train_acc = []
        fold_metrics_KAN_test_acc = []
        fold_metrics_KAN_train_loss = []
        fold_metrics_KAN_test_loss = []
        fold_metrics_MLP_train_acc = []
        fold_metrics_MLP_test_acc = []
        fold_metrics_MLP_train_loss = []
        fold_metrics_MLP_test_loss = []

        for epoch in range(epochs):
            print(f'###### EPOCH {epoch} ######')

            # Train KAN
            start_time_KAN = time.time()
            results_kan = kan_model.train_model(dataset, opt="LBFGS", steps=steps, loss_fn=loss_fn, batch=batch, lr=lr)
            end_time_KAN = time.time()

            fold_metrics_KAN_train_acc.append(np.mean(results_kan['train_acc']))
            fold_metrics_KAN_test_acc.append(np.mean(results_kan['test_acc']))
            fold_metrics_KAN_train_loss.append(np.mean(results_kan['train_loss']))
            fold_metrics_KAN_test_loss.append(np.mean(results_kan['test_loss']))

            # Update training time for KAN
            metrics_KAN[subject]['train_time'] += end_time_KAN - start_time_KAN

            # Train MLP
            start_time_MLP = time.time()
            results_mlp = mlp_model.train(dataset, opt="LBFGS", steps=steps, loss_fn=loss_fn, batch=batch, lr=lr)
            end_time_MLP = time.time()

            fold_metrics_MLP_train_acc.append(np.mean(results_mlp['train_acc']))
            fold_metrics_MLP_test_acc.append(np.mean(results_mlp['test_acc']))
            fold_metrics_MLP_train_loss.append(np.mean(results_mlp['train_loss']))
            fold_metrics_MLP_test_loss.append(np.mean(results_mlp['test_loss']))

            # Update training time for MLP
            metrics_MLP[subject]['train_time'] += end_time_MLP - start_time_MLP

        # Store fold results
        metrics_KAN[subject]['train_acc'].append(fold_metrics_KAN_train_acc)
        metrics_KAN[subject]['test_acc'].append(fold_metrics_KAN_test_acc)
        metrics_KAN[subject]['train_loss'].append(fold_metrics_KAN_train_loss)
        metrics_KAN[subject]['test_loss'].append(fold_metrics_KAN_test_loss)

        metrics_MLP[subject]['train_acc'].append(fold_metrics_MLP_train_acc)
        metrics_MLP[subject]['test_acc'].append(fold_metrics_MLP_test_acc)
        metrics_MLP[subject]['train_loss'].append(fold_metrics_MLP_train_loss)
        metrics_MLP[subject]['test_loss'].append(fold_metrics_MLP_test_loss)

# Print out total training time for KAN and MLP for each subject
for subject in subjects:
    print(f"### Subject {subject} ###")
    
    # Training time
    print(f"Training time KAN: {metrics_KAN[subject]['train_time']:.2f} seconds")
    print(f"Training time MLP: {metrics_MLP[subject]['train_time']:.2f} seconds")
    
    # Training Accuracy and Loss (KAN)
    print("KAN Training Accuracy per Fold:", [np.mean(acc) for acc in metrics_KAN[subject]['train_acc']])
    print("KAN Training Loss per Fold:", [np.mean(loss) for loss in metrics_KAN[subject]['train_loss']])
    
    # Testing Accuracy and Loss (KAN)
    print("KAN Testing Accuracy per Fold:", [np.mean(acc) for acc in metrics_KAN[subject]['test_acc']])
    print("KAN Testing Loss per Fold:", [np.mean(loss) for loss in metrics_KAN[subject]['test_loss']])
    
    # Training Accuracy and Loss (MLP)
    print("MLP Training Accuracy per Fold:", [np.mean(acc) for acc in metrics_MLP[subject]['train_acc']])
    print("MLP Training Loss per Fold:", [np.mean(loss) for loss in metrics_MLP[subject]['train_loss']])
    
    # Testing Accuracy and Loss (MLP)
    print("MLP Testing Accuracy per Fold:", [np.mean(acc) for acc in metrics_MLP[subject]['test_acc']])
    print("MLP Testing Loss per Fold:", [np.mean(loss) for loss in metrics_MLP[subject]['test_loss']])


### Subject 1 ###
checkpoint directory created: ./model
saving model version 0.0
--- Fold 1 ---
###### EPOCH 0 ######


train acc: 0.875 | test acc: 0.562 : 100%|██████████████████████████| 50/50 [00:23<00:00,  2.14it/s]
train acc: 0.875 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:02<00:00, 23.55it/s]


###### EPOCH 1 ######


train acc: 1.000 | test acc: 0.312 : 100%|██████████████████████████| 50/50 [00:23<00:00,  2.12it/s]
train acc: 0.812 | test acc: 0.438 : 100%|██████████████████████████| 50/50 [00:02<00:00, 24.22it/s]


###### EPOCH 2 ######


...

###### EPOCH 18 ######


train acc: 1.000 | test acc: 0.562 : 100%|██████████████████████████| 50/50 [00:28<00:00,  1.73it/s]
train acc: 0.750 | test acc: 0.625 : 100%|██████████████████████████| 50/50 [00:02<00:00, 22.39it/s]


###### EPOCH 19 ######


train acc: 0.812 | test acc: 0.438 : 100%|██████████████████████████| 50/50 [00:30<00:00,  1.65it/s]
train acc: 0.625 | test acc: 0.500 : 100%|██████████████████████████| 50/50 [00:02<00:00, 23.03it/s]

### Subject 1 ###
Training time KAN: 5100.50 seconds
Training time MLP: 436.45 seconds
KAN Training Accuracy per Fold: [0.9193125, 0.91875, 0.9126875, 0.903625, 0.90900004, 0.8998124, 0.8791876, 0.88706255, 0.90412503, 0.9045]
KAN Training Loss per Fold: [1.0775998239904996, 3.3698837312108765, 18.148739364014286, 38.1076228030173, 37.44467269949537, 86.68865788148733, 124.47148454218079, 151.46548976312667, 227.47289163268223, 323.48204420192525]
KAN Testing Accuracy per Fold: [0.5033125, 0.5010625, 0.50124997, 0.49599996, 0.5018125, 0.5055, 0.49718753, 0.500875, 0.49437505, 0.502375]
KAN Testing Loss per Fold: [9.98402926306525, 36.634116077233216, 191.30533985361143, 388.8614350715878, 363.97304704014994, 826.037063898332, 923.5253455268237, 1113.2775506197477, 2016.2304968536039, 2654.0952320131314]
MLP Training Accuracy per Fold: [0.833125, 0.773125, 0.7359375, 0.7275, 0.72475, 0.7226875, 0.7150625, 0.72218746, 0.7163125, 0.71818745]
MLP Training Loss per Fold: [0.5639043516108371

In [8]:
# TWO_CLASSRAW
metrics_KAN = {}
metrics_MLP = {}
total_train_time_KAN = {}
total_train_time_MLP = {}

subjects = [4,5,6]  # Add all 9 subjects here

for subject in subjects:
    print(f"### Subject {subject} ###")

    # Get data for each subject
    epochs_EEG = mne.concatenate_epochs(data_EEG(subjects=[subject]))
    df = epochs_EEG.to_data_frame().drop(columns=['time', 'epoch'])
    X = df.drop(columns=['condition']).values
    y = df.condition.values
    y = le.fit_transform(y)

    X = torch.from_numpy(X).double()
    y = torch.from_numpy(y).long()

    input_layer = X.shape[1]
    output_layer = len(np.unique(y))

    kan_model = KANModel(input_shape=input_layer, output_shape=output_layer, device=device)
    mlp_model = MLP(input_layer, hidden_layers, output_layer).to(device)

    for param in mlp_model.parameters():
        param.data = param.data.double()

    kan_optimizer = LBFGS(kan_model.parameters(), lr=lr)
    mlp_optimizer = LBFGS(mlp_model.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe")

    metrics_KAN[subject] = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'test_loss': [], 'train_time': 0}
    metrics_MLP[subject] = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'test_loss': [], 'train_time': 0}

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        dataset = {
            'train_input': X_train,
            'train_label': y_train,
            'test_input': X_test,
            'test_label': y_test
        }

        # Train KAN
        print(f"--- Fold {fold + 1} ---")

        fold_metrics_KAN_train_acc = []
        fold_metrics_KAN_test_acc = []
        fold_metrics_KAN_train_loss = []
        fold_metrics_KAN_test_loss = []
        fold_metrics_MLP_train_acc = []
        fold_metrics_MLP_test_acc = []
        fold_metrics_MLP_train_loss = []
        fold_metrics_MLP_test_loss = []

        for epoch in range(epochs):
            print(f'###### EPOCH {epoch} ######')

            # Train KAN
            start_time_KAN = time.time()
            results_kan = kan_model.train_model(dataset, opt="LBFGS", steps=steps, loss_fn=loss_fn, batch=batch, lr=lr)
            end_time_KAN = time.time()

            fold_metrics_KAN_train_acc.append(np.mean(results_kan['train_acc']))
            fold_metrics_KAN_test_acc.append(np.mean(results_kan['test_acc']))
            fold_metrics_KAN_train_loss.append(np.mean(results_kan['train_loss']))
            fold_metrics_KAN_test_loss.append(np.mean(results_kan['test_loss']))

            # Update training time for KAN
            metrics_KAN[subject]['train_time'] += end_time_KAN - start_time_KAN

            # Train MLP
            start_time_MLP = time.time()
            results_mlp = mlp_model.train(dataset, opt="LBFGS", steps=steps, loss_fn=loss_fn, batch=batch, lr=lr)
            end_time_MLP = time.time()

            fold_metrics_MLP_train_acc.append(np.mean(results_mlp['train_acc']))
            fold_metrics_MLP_test_acc.append(np.mean(results_mlp['test_acc']))
            fold_metrics_MLP_train_loss.append(np.mean(results_mlp['train_loss']))
            fold_metrics_MLP_test_loss.append(np.mean(results_mlp['test_loss']))

            # Update training time for MLP
            metrics_MLP[subject]['train_time'] += end_time_MLP - start_time_MLP

        # Store fold results
        metrics_KAN[subject]['train_acc'].append(fold_metrics_KAN_train_acc)
        metrics_KAN[subject]['test_acc'].append(fold_metrics_KAN_test_acc)
        metrics_KAN[subject]['train_loss'].append(fold_metrics_KAN_train_loss)
        metrics_KAN[subject]['test_loss'].append(fold_metrics_KAN_test_loss)

        metrics_MLP[subject]['train_acc'].append(fold_metrics_MLP_train_acc)
        metrics_MLP[subject]['test_acc'].append(fold_metrics_MLP_test_acc)
        metrics_MLP[subject]['train_loss'].append(fold_metrics_MLP_train_loss)
        metrics_MLP[subject]['test_loss'].append(fold_metrics_MLP_test_loss)

# Print out total training time for KAN and MLP for each subject
for subject in subjects:
    print(f"### Subject {subject} ###")
    
    # Training time
    print(f"Training time KAN: {metrics_KAN[subject]['train_time']:.2f} seconds")
    print(f"Training time MLP: {metrics_MLP[subject]['train_time']:.2f} seconds")
    
    # Training Accuracy and Loss (KAN)
    print("KAN Training Accuracy per Fold:", [np.mean(acc) for acc in metrics_KAN[subject]['train_acc']])
    print("KAN Training Loss per Fold:", [np.mean(loss) for loss in metrics_KAN[subject]['train_loss']])
    
    # Testing Accuracy and Loss (KAN)
    print("KAN Testing Accuracy per Fold:", [np.mean(acc) for acc in metrics_KAN[subject]['test_acc']])
    print("KAN Testing Loss per Fold:", [np.mean(loss) for loss in metrics_KAN[subject]['test_loss']])
    
    # Training Accuracy and Loss (MLP)
    print("MLP Training Accuracy per Fold:", [np.mean(acc) for acc in metrics_MLP[subject]['train_acc']])
    print("MLP Training Loss per Fold:", [np.mean(loss) for loss in metrics_MLP[subject]['train_loss']])
    
    # Testing Accuracy and Loss (MLP)
    print("MLP Testing Accuracy per Fold:", [np.mean(acc) for acc in metrics_MLP[subject]['test_acc']])
    print("MLP Testing Loss per Fold:", [np.mean(loss) for loss in metrics_MLP[subject]['test_loss']])


### Subject 4 ###
checkpoint directory created: ./model
saving model version 0.0
--- Fold 1 ---
###### EPOCH 0 ######


train acc: 0.938 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:23<00:00,  2.14it/s]
train acc: 0.938 | test acc: 0.312 : 100%|██████████████████████████| 50/50 [00:02<00:00, 22.54it/s]


###### EPOCH 1 ######


train acc: 1.000 | test acc: 0.500 : 100%|██████████████████████████| 50/50 [00:23<00:00,  2.17it/s]
train acc: 0.938 | test acc: 0.562 : 100%|██████████████████████████| 50/50 [00:02<00:00, 23.21it/s]


###### EPOCH 2 ######


...

###### EPOCH 18 ######


train acc: 0.875 | test acc: 0.625 : 100%|██████████████████████████| 50/50 [00:30<00:00,  1.66it/s]
train acc: 0.875 | test acc: 0.500 : 100%|██████████████████████████| 50/50 [00:02<00:00, 23.08it/s]


###### EPOCH 19 ######


train acc: 0.875 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:27<00:00,  1.82it/s]
train acc: 0.875 | test acc: 0.500 : 100%|██████████████████████████| 50/50 [00:02<00:00, 23.01it/s]

### Subject 4 ###
Training time KAN: 4921.93 seconds
Training time MLP: 442.80 seconds
KAN Training Accuracy per Fold: [0.91824996, 0.9213125, 0.9203124, 0.9118125, 0.9178125, 0.90768754, 0.903, 0.90812504, 0.91175, 0.9021249]
KAN Training Loss per Fold: [1.1844144475226528, 3.872770830285513, 9.010513986286075, 18.13119273706701, 34.972774509277755, 85.90849627767696, 225.12219671424663, 399.25270416882165, 197.817876971576, 238.8326742926434]
KAN Testing Accuracy per Fold: [0.49643755, 0.49925, 0.49300003, 0.49631253, 0.5019375, 0.49962497, 0.49837503, 0.49725002, 0.4966875, 0.50487506]
KAN Testing Loss per Fold: [10.96815731923314, 42.25186713545394, 99.6196375745555, 188.4572632469032, 391.0655947630461, 805.4036700378373, 1802.0380718355161, 3617.135656354264, 1759.1192158318383, 1938.9221881225428]
MLP Training Accuracy per Fold: [0.8150624, 0.74593747, 0.7288125, 0.73687494, 0.73675, 0.736625, 0.73475, 0.717625, 0.7139375, 0.72012496]
MLP Training Loss per Fold: [0.6917926683421

In [10]:
# TWO_CLASSRAW
metrics_KAN = {}
metrics_MLP = {}
total_train_time_KAN = {}
total_train_time_MLP = {}

subjects = [7,8,9]  # Add all 9 subjects here

for subject in subjects:
    print(f"### Subject {subject} ###")

    # Get data for each subject
    epochs_EEG = mne.concatenate_epochs(data_EEG(subjects=[subject]))
    df = epochs_EEG.to_data_frame().drop(columns=['time', 'epoch'])
    X = df.drop(columns=['condition']).values
    y = df.condition.values
    y = le.fit_transform(y)

    X = torch.from_numpy(X).double()
    y = torch.from_numpy(y).long()

    input_layer = X.shape[1]
    output_layer = len(np.unique(y))

    kan_model = KANModel(input_shape=input_layer, output_shape=output_layer, device=device)
    mlp_model = MLP(input_layer, hidden_layers, output_layer).to(device)

    for param in mlp_model.parameters():
        param.data = param.data.double()

    kan_optimizer = LBFGS(kan_model.parameters(), lr=lr)
    mlp_optimizer = LBFGS(mlp_model.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe")

    metrics_KAN[subject] = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'test_loss': [], 'train_time': 0}
    metrics_MLP[subject] = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'test_loss': [], 'train_time': 0}

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        dataset = {
            'train_input': X_train,
            'train_label': y_train,
            'test_input': X_test,
            'test_label': y_test
        }

        # Train KAN
        print(f"--- Fold {fold + 1} ---")

        fold_metrics_KAN_train_acc = []
        fold_metrics_KAN_test_acc = []
        fold_metrics_KAN_train_loss = []
        fold_metrics_KAN_test_loss = []
        fold_metrics_MLP_train_acc = []
        fold_metrics_MLP_test_acc = []
        fold_metrics_MLP_train_loss = []
        fold_metrics_MLP_test_loss = []

        for epoch in range(epochs):
            print(f'###### EPOCH {epoch} ######')

            # Train KAN
            start_time_KAN = time.time()
            results_kan = kan_model.train_model(dataset, opt="LBFGS", steps=steps, loss_fn=loss_fn, batch=batch, lr=lr)
            end_time_KAN = time.time()

            fold_metrics_KAN_train_acc.append(np.mean(results_kan['train_acc']))
            fold_metrics_KAN_test_acc.append(np.mean(results_kan['test_acc']))
            fold_metrics_KAN_train_loss.append(np.mean(results_kan['train_loss']))
            fold_metrics_KAN_test_loss.append(np.mean(results_kan['test_loss']))

            # Update training time for KAN
            metrics_KAN[subject]['train_time'] += end_time_KAN - start_time_KAN

            # Train MLP
            start_time_MLP = time.time()
            results_mlp = mlp_model.train(dataset, opt="LBFGS", steps=steps, loss_fn=loss_fn, batch=batch, lr=lr)
            end_time_MLP = time.time()

            fold_metrics_MLP_train_acc.append(np.mean(results_mlp['train_acc']))
            fold_metrics_MLP_test_acc.append(np.mean(results_mlp['test_acc']))
            fold_metrics_MLP_train_loss.append(np.mean(results_mlp['train_loss']))
            fold_metrics_MLP_test_loss.append(np.mean(results_mlp['test_loss']))

            # Update training time for MLP
            metrics_MLP[subject]['train_time'] += end_time_MLP - start_time_MLP

        # Store fold results
        metrics_KAN[subject]['train_acc'].append(fold_metrics_KAN_train_acc)
        metrics_KAN[subject]['test_acc'].append(fold_metrics_KAN_test_acc)
        metrics_KAN[subject]['train_loss'].append(fold_metrics_KAN_train_loss)
        metrics_KAN[subject]['test_loss'].append(fold_metrics_KAN_test_loss)

        metrics_MLP[subject]['train_acc'].append(fold_metrics_MLP_train_acc)
        metrics_MLP[subject]['test_acc'].append(fold_metrics_MLP_test_acc)
        metrics_MLP[subject]['train_loss'].append(fold_metrics_MLP_train_loss)
        metrics_MLP[subject]['test_loss'].append(fold_metrics_MLP_test_loss)

# Print out total training time for KAN and MLP for each subject
for subject in subjects:
    print(f"### Subject {subject} ###")
    
    # Training time
    print(f"Training time KAN: {metrics_KAN[subject]['train_time']:.2f} seconds")
    print(f"Training time MLP: {metrics_MLP[subject]['train_time']:.2f} seconds")
    
    # Training Accuracy and Loss (KAN)
    print("KAN Training Accuracy per Fold:", [np.mean(acc) for acc in metrics_KAN[subject]['train_acc']])
    print("KAN Training Loss per Fold:", [np.mean(loss) for loss in metrics_KAN[subject]['train_loss']])
    
    # Testing Accuracy and Loss (KAN)
    print("KAN Testing Accuracy per Fold:", [np.mean(acc) for acc in metrics_KAN[subject]['test_acc']])
    print("KAN Testing Loss per Fold:", [np.mean(loss) for loss in metrics_KAN[subject]['test_loss']])
    
    # Training Accuracy and Loss (MLP)
    print("MLP Training Accuracy per Fold:", [np.mean(acc) for acc in metrics_MLP[subject]['train_acc']])
    print("MLP Training Loss per Fold:", [np.mean(loss) for loss in metrics_MLP[subject]['train_loss']])
    
    # Testing Accuracy and Loss (MLP)
    print("MLP Testing Accuracy per Fold:", [np.mean(acc) for acc in metrics_MLP[subject]['test_acc']])
    print("MLP Testing Loss per Fold:", [np.mean(loss) for loss in metrics_MLP[subject]['test_loss']])


### Subject 7 ###
checkpoint directory created: ./model
saving model version 0.0
--- Fold 1 ---
###### EPOCH 0 ######


train acc: 1.000 | test acc: 0.625 : 100%|██████████████████████████| 50/50 [00:23<00:00,  2.17it/s]
train acc: 0.938 | test acc: 0.438 : 100%|██████████████████████████| 50/50 [00:02<00:00, 22.75it/s]


###### EPOCH 1 ######


train acc: 0.938 | test acc: 0.250 : 100%|██████████████████████████| 50/50 [00:22<00:00,  2.20it/s]
train acc: 0.938 | test acc: 0.562 : 100%|██████████████████████████| 50/50 [00:02<00:00, 23.27it/s]


###### EPOCH 2 ######


...

###### EPOCH 18 ######


train acc: 0.938 | test acc: 0.500 : 100%|██████████████████████████| 50/50 [00:29<00:00,  1.68it/s]
train acc: 0.625 | test acc: 0.438 : 100%|██████████████████████████| 50/50 [00:02<00:00, 22.88it/s]


###### EPOCH 19 ######


train acc: 0.562 | test acc: 0.688 : 100%|██████████████████████████| 50/50 [00:27<00:00,  1.84it/s]
train acc: 0.750 | test acc: 0.438 : 100%|██████████████████████████| 50/50 [00:02<00:00, 18.88it/s]

### Subject 7 ###
Training time KAN: 5051.02 seconds
Training time MLP: 438.61 seconds
KAN Training Accuracy per Fold: [0.92275, 0.92406255, 0.92775, 0.8961875, 0.90181255, 0.90787494, 0.90174997, 0.90062505, 0.9145626, 0.90893734]
KAN Training Loss per Fold: [2.139146756532981, 9.105711760644706, 15.365260458382874, 57.55106854438757, 78.99033376554782, 206.8111477357082, 421.9457674270194, 387.2278203467888, 517.012663404502, 519.3376218132955]
KAN Testing Accuracy per Fold: [0.5005625, 0.51112497, 0.50481254, 0.4986875, 0.49812502, 0.50250006, 0.49831247, 0.5026875, 0.50606245, 0.49993747]
KAN Testing Loss per Fold: [20.955589156040027, 101.02063103645109, 178.4758183760494, 461.0606197895904, 656.1986215347481, 1608.6273440749915, 3221.5136143218056, 3009.1081831647084, 4870.189566372674, 4628.404209475229]
MLP Training Accuracy per Fold: [0.8026875, 0.7531249, 0.72431254, 0.7025, 0.7036251, 0.69806254, 0.7028125, 0.70212495, 0.6869375, 0.698125]
MLP Training Loss per Fold: [0.6126

In [9]:
# MULTICLASS_RAW
metrics_KAN = {}
metrics_MLP = {}
total_train_time_KAN = {}
total_train_time_MLP = {}

subjects = [1,2,3]  # Add all 9 subjects here

for subject in subjects:
    print(f"### Subject {subject} ###")

    # Get data for each subject
    epochs_EEG = mne.concatenate_epochs(data_EEG(subjects=[subject],multiclass=True))
    df = epochs_EEG.to_data_frame().drop(columns=['time', 'epoch'])
    X = df.drop(columns=['condition']).values
    y = df.condition.values
    y = le.fit_transform(y)

    X = torch.from_numpy(X).double()
    y = torch.from_numpy(y).long()

    input_layer = X.shape[1]
    output_layer = len(np.unique(y))

    kan_model = KANModel(input_shape=input_layer, output_shape=output_layer, device=device)
    mlp_model = MLP(input_layer, hidden_layers, output_layer).to(device)

    for param in mlp_model.parameters():
        param.data = param.data.double()

    kan_optimizer = LBFGS(kan_model.parameters(), lr=lr)
    mlp_optimizer = LBFGS(mlp_model.parameters(), lr=lr, history_size=10, line_search_fn="strong_wolfe")

    metrics_KAN[subject] = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'test_loss': [], 'train_time': 0}
    metrics_MLP[subject] = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'test_loss': [], 'train_time': 0}

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        dataset = {
            'train_input': X_train,
            'train_label': y_train,
            'test_input': X_test,
            'test_label': y_test
        }

        # Train KAN
        print(f"--- Fold {fold + 1} ---")

        fold_metrics_KAN_train_acc = []
        fold_metrics_KAN_test_acc = []
        fold_metrics_KAN_train_loss = []
        fold_metrics_KAN_test_loss = []
        fold_metrics_MLP_train_acc = []
        fold_metrics_MLP_test_acc = []
        fold_metrics_MLP_train_loss = []
        fold_metrics_MLP_test_loss = []

        for epoch in range(epochs):
            print(f'###### EPOCH {epoch} ######')

            # Train KAN
            start_time_KAN = time.time()
            results_kan = kan_model.train_model(dataset, opt="LBFGS", steps=steps, loss_fn=loss_fn, batch=batch, lr=lr)
            end_time_KAN = time.time()

            fold_metrics_KAN_train_acc.append(np.mean(results_kan['train_acc']))
            fold_metrics_KAN_test_acc.append(np.mean(results_kan['test_acc']))
            fold_metrics_KAN_train_loss.append(np.mean(results_kan['train_loss']))
            fold_metrics_KAN_test_loss.append(np.mean(results_kan['test_loss']))

            # Update training time for KAN
            metrics_KAN[subject]['train_time'] += end_time_KAN - start_time_KAN

            # Train MLP
            start_time_MLP = time.time()
            results_mlp = mlp_model.train(dataset, opt="LBFGS", steps=steps, loss_fn=loss_fn, batch=batch, lr=lr)
            end_time_MLP = time.time()

            fold_metrics_MLP_train_acc.append(np.mean(results_mlp['train_acc']))
            fold_metrics_MLP_test_acc.append(np.mean(results_mlp['test_acc']))
            fold_metrics_MLP_train_loss.append(np.mean(results_mlp['train_loss']))
            fold_metrics_MLP_test_loss.append(np.mean(results_mlp['test_loss']))

            # Update training time for MLP
            metrics_MLP[subject]['train_time'] += end_time_MLP - start_time_MLP

        # Store fold results
        metrics_KAN[subject]['train_acc'].append(fold_metrics_KAN_train_acc)
        metrics_KAN[subject]['test_acc'].append(fold_metrics_KAN_test_acc)
        metrics_KAN[subject]['train_loss'].append(fold_metrics_KAN_train_loss)
        metrics_KAN[subject]['test_loss'].append(fold_metrics_KAN_test_loss)

        metrics_MLP[subject]['train_acc'].append(fold_metrics_MLP_train_acc)
        metrics_MLP[subject]['test_acc'].append(fold_metrics_MLP_test_acc)
        metrics_MLP[subject]['train_loss'].append(fold_metrics_MLP_train_loss)
        metrics_MLP[subject]['test_loss'].append(fold_metrics_MLP_test_loss)

# Print out total training time for KAN and MLP for each subject
# Print out total training time, accuracy, and loss for KAN and MLP for each subject
for subject in subjects:
    print(f"### Subject {subject} ###")
    
    # Training time
    print(f"Training time KAN: {metrics_KAN[subject]['train_time']:.2f} seconds")
    print(f"Training time MLP: {metrics_MLP[subject]['train_time']:.2f} seconds")
    
    # Training Accuracy and Loss (KAN)
    print("KAN Training Accuracy per Fold:", [np.mean(acc) for acc in metrics_KAN[subject]['train_acc']])
    print("KAN Training Loss per Fold:", [np.mean(loss) for loss in metrics_KAN[subject]['train_loss']])
    
    # Testing Accuracy and Loss (KAN)
    print("KAN Testing Accuracy per Fold:", [np.mean(acc) for acc in metrics_KAN[subject]['test_acc']])
    print("KAN Testing Loss per Fold:", [np.mean(loss) for loss in metrics_KAN[subject]['test_loss']])
    
    # Training Accuracy and Loss (MLP)
    print("MLP Training Accuracy per Fold:", [np.mean(acc) for acc in metrics_MLP[subject]['train_acc']])
    print("MLP Training Loss per Fold:", [np.mean(loss) for loss in metrics_MLP[subject]['train_loss']])
    
    # Testing Accuracy and Loss (MLP)
    print("MLP Testing Accuracy per Fold:", [np.mean(acc) for acc in metrics_MLP[subject]['test_acc']])
    print("MLP Testing Loss per Fold:", [np.mean(loss) for loss in metrics_MLP[subject]['test_loss']])


### Subject 1 ###
checkpoint directory created: ./model
saving model version 0.0
--- Fold 1 ---
###### EPOCH 0 ######


train acc: 0.750 | test acc: 0.312 : 100%|██████████████████████████| 50/50 [00:40<00:00,  1.24it/s]
train acc: 0.938 | test acc: 0.250 : 100%|██████████████████████████| 50/50 [00:02<00:00, 20.37it/s]


###### EPOCH 1 ######


train acc: 0.875 | test acc: 0.250 : 100%|██████████████████████████| 50/50 [00:39<00:00,  1.27it/s]
train acc: 0.812 | test acc: 0.125 : 100%|██████████████████████████| 50/50 [00:02<00:00, 19.88it/s]


###### EPOCH 2 ######


...

###### EPOCH 18 ######


train acc: 0.375 | test acc: 0.250 : 100%|██████████████████████████| 50/50 [00:47<00:00,  1.06it/s]
train acc: 0.438 | test acc: 0.125 : 100%|██████████████████████████| 50/50 [00:02<00:00, 18.06it/s]


###### EPOCH 19 ######


train acc: 0.625 | test acc: 0.250 : 100%|██████████████████████████| 50/50 [00:40<00:00,  1.23it/s]
train acc: 0.375 | test acc: 0.375 : 100%|██████████████████████████| 50/50 [00:02<00:00, 21.10it/s]


### Subject 1 ###
Training time KAN: 8086.05 seconds
Training time MLP: 494.17 seconds
KAN Training Accuracy per Fold: [0.78256255, 0.80474997, 0.82824993, 0.8256875, 0.82875, 0.82618743, 0.83843744, 0.8376875, 0.8515624, 0.8368125]
KAN Training Loss per Fold: [0.8325836108349373, 1.1416168886786189, 1.2494003316157727, 1.769578900303961, 1.7113002411342442, 1.6772504378342634, 2.2136595055120347, 3.089926183100711, 3.8933260910057106, 3.2149357492108583]
KAN Testing Accuracy per Fold: [0.2526875, 0.2535, 0.24887499, 0.25237498, 0.256375, 0.25606254, 0.25225002, 0.2485625, 0.25562504, 0.2490625]
KAN Testing Loss per Fold: [3.5970310951112325, 6.23789606833701, 7.916581918369488, 11.065906427090347, 11.311271951487539, 10.35693847082026, 15.215277271899003, 22.54254820670695, 30.903964146348926, 22.568740330459548]
MLP Training Accuracy per Fold: [0.55706257, 0.4945, 0.46199998, 0.46375003, 0.45406246, 0.44675, 0.45025, 0.44881248, 0.46062502, 0.46318755]
MLP Training Loss per Fold: [0.